# TensorFlow APIs

---

### Listing files in a directory: Plain Python (off-graph)

In [ ]:
import glob

glob.glob('my_dir/*')

### Listing files in a directory: TensorFlow ops (on-graph)

In [ ]:
import tensorflow as tf

glob = tf.matching_files('my_dir/*')

session = tf.Session()
session.run(glob)

For more, see: [Everything in the Graph? Even Glob?](http://planspace.org/20170428-everything_in_the_graph_even_glob/)

For an example of TensorFlow functionality with off-graph and on-graph alternatives, see:
 * [TFRecords for Humans](http://planspace.org/20170323-tfrecords_for_humans/) (off-graph)
 * [Parsing TFRecords inside the TensorFlow Graph](http://planspace.org/20170426-parsing_tfrecords_inside_the_tensorflow_graph/) (on-graph)

---

### Making a Convolutional Layer: Graph Op-Level

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])

weight_initial = tf.truncated_normal([5, 5, 1, 32], stddev=0.1)
weight = tf.Variable(weight_initial)
bias_initial = tf.constant(0.1, shape=[32])
bias = tf.Variable(bias_initial)

convolved = tf.nn.conv2d(x, weight, strides=[1, 1, 1, 1], padding='SAME')
convolved_biased = convolved + bias
y = tf.nn.relu(convolved_biased)

Based on: [Build a Multilayer Convolutional Network](https://www.tensorflow.org/get_started/mnist/pros#build_a_multilayer_convolutional_network)

### Making a Convolutional Layer: Layers

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])

y = tf.layers.conv2d(inputs=x, 
                     filters=32, kernel_size=[5, 5], padding='same', 
                     activation=tf.nn.relu)

Based on: [A Guide to TF Layers: Building a Convolutional Neural Network](https://www.tensorflow.org/tutorials/layers)

### Making a Convolutional Layer: Keras Functional

In [ ]:
x = tf.contrib.keras.layers.Input(shape=[28, 28, 1])

y = tf.contrib.keras.layers.Conv2D(filters=32, kernel_size=[5, 5], padding='same',
                                   activation='relu')(x)

### Making a Convolutional Layer: Keras `Sequential`

In [ ]:
model = tf.contrib.keras.models.Sequential()

model.add(tf.contrib.keras.layers.Conv2D(filters=32, kernel_size=[5, 5], padding='same',
                                         activation='relu', 
                                         input_shape=[28, 28, 1]))

### More advanced TensorFlow Keras

In [ ]:
x = tf.contrib.keras.layers.Input(shape=[28, 28, 1])
residual = x
x = tf.contrib.keras.layers.Activation('relu')(x)
x = tf.contrib.keras.layers.SeparableConv2D(filters=728, kernel_size=[3, 3], padding='same')(x)
x = tf.contrib.keras.layers.BatchNormalization()(x)
y = tf.contrib.keras.layers.add([x, residual])

This is a cartoon of a section from: [xception.py](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/keras/python/keras/applications/xception.py)

See also: [Xception: Deep Learning with Depthwise Separable Convolutions](https://arxiv.org/abs/1610.02357)

### Expected around TensorFlow 1.3

In [ ]:
# estimator = keras_model.get_estimator()

---

### Simple regression example

Predicting annualized US GDP growth from presidential party. ([data](http://planspace.org/20170505-simple_dataset_us_presidential_party_and_gdp_growth/))

In [ ]:
!head -3 president_gdp.csv

In [ ]:
import numpy as np
import pandas as pd

data = pd.read_csv('president_gdp.csv')
party = data.party == 'D'
party = np.expand_dims(party, axis=1)
growth = data.growth

### Simple regression with `sklearn`

In [ ]:
import sklearn.linear_model

model = sklearn.linear_model.LinearRegression()
model.fit(X=party, y=growth)
model.predict([[0], [1]])

### Simple regression with a TensorFlow canned Estimator

In [ ]:
# clear old run...
import shutil
shutil.rmtree('tflinreg', ignore_errors=True)

In [ ]:
party_col = tf.contrib.layers.real_valued_column(column_name='')

In [ ]:
model = tf.contrib.learn.LinearRegressor(feature_columns=[party_col],
                                         model_dir='tflinreg')

In [ ]:
model.fit(x=party, y=growth, steps=1000)
list(model.predict(np.array([[0], [1]])))

In [ ]:
!tensorboard --logdir tflinreg

After startup, go to http://localhost:6006/ to see the interface.

To continue with the notebook, interrupt the kernel by using the square "stop" button or by typing esc, i, i.

For more, see:
 * [Simple Regression with a TensorFlow Estimator](http://planspace.org/20170506-simple_regression_with_a_tensorflow_estimator/)
 * [Navigating TensorFlow Estimator Documentation](http://planspace.org/20170521-navigating_tensorflow_estimator_documentation/)
 *  Feature columns, input functions, model functions, experiments, `learn_runner`, and more!

---

### The `Dataset` API

The comparison to the low-level way of doing this is a little involved; for more, see:
 * [TensorFlow documentation on an 8-step data pipeline](https://www.tensorflow.org/programmers_guide/reading_data#reading_from_files)
 * [TensorFlow and Queues](http://planspace.org/20170327-tensorflow_and_queues/)
 * [The TensorFlow Coordinator for Python Threading](http://planspace.org/20170324-the_tensorflow_coordinator_for_python_threading/)
 * [TensorFlow's QueueRunner](http://planspace.org/20170430-tensorflows_queuerunner/)

In [ ]:
dataset = tf.contrib.data.TextLineDataset('president_gdp.csv')
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

session = tf.Session()
session.run(next_element)

In [ ]:
session.run(next_element)

In [ ]:
session.run(next_element)

For more, see: [Using the `Dataset` API for TensorFlow Input Pipelines](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/data)

---

### Using a pre-trained model with Keras in TensorFlow

In [ ]:
model = tf.contrib.keras.applications.InceptionV3()

In [ ]:
filename = 'n01882714_4157_koala_bear.jpg'
image = tf.contrib.keras.preprocessing.image.load_img(
    filename, target_size=(299, 299))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(image)

In [ ]:
array = tf.contrib.keras.preprocessing.image.img_to_array(image)
array = np.expand_dims(array, axis=0)
array = tf.contrib.keras.applications.inception_v3.preprocess_input(array)

In [ ]:
probabilities = model.predict(array)

In [ ]:
tf.contrib.keras.applications.inception_v3.decode_predictions(probabilities)

For more, see: [Pre-trained Models with Keras in TensorFlow](http://planspace.org/20170502-canned_models_with_keras_in_tensorflow/)